# Posthoc Classifier

## Install, Paths and Parameters

In [1]:
# This extension reloads external Python files
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import math
import torch
from torch import nn
from tqdm import tqdm
import random
import sys
from torch.utils.data import random_split
from matplotlib import pyplot as plt

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino
from src.model.data import create_loader, adv_dataset, ORIGINAL_TRANSFORM, NO_NORM_TRANSFORM, TO_TENSOR_TRANSFORM

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')
# Path for intermediate outputs
BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc/')

# Original Dataset
ORI_PATH = Path(DATA_PATH, 'ori_data/')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

TR_PATH = Path(ORI_PATH, 'train/')
TR_ORI_LABEL_PATH = Path(TR_PATH,'correct_labels.txt')
TR_ORI_IMAGES_PATH = Path(TR_PATH,'images')

VAL_PATH = Path(ORI_PATH, 'validation/')
VAL_ORI_LABEL_PATH = Path(VAL_PATH,'correct_labels.txt')
VAL_ORI_IMAGES_PATH = Path(VAL_PATH,'images')

# DAmageNet
#DN_PATH = Path(DATA_PATH, 'damageNet')
#DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
#DN_IMAGES_PATH = Path(DN_PATH, 'images')
#DN_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'damagenet')
#DN_POSTHOC_LABEL_PATH = Path(DN_POSTHOC_PATH, 'labels.csv')

# PGD
VAL_PGD_PATH = Path(DATA_PATH, 'adversarial_data/pgd_03/validation')
VAL_PGD_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_PGD_IMAGES_PATH = Path(VAL_PGD_PATH, 'images')
VAL_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/validation/')
VAL_PGD_POSTHOC_LABEL_PATH = Path(VAL_PGD_POSTHOC_PATH, 'labels.csv')

# CW
VAL_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/validation')
VAL_CW_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_CW_IMAGES_PATH = Path(VAL_CW_PATH, 'images')
VAL_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/validation/')
VAL_CW_POSTHOC_LABEL_PATH = Path(VAL_CW_POSTHOC_PATH, 'labels.csv')

# FGSM
TR_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/train')
TR_FGSM_LABEL_PATH = TR_ORI_LABEL_PATH
TR_FGSM_IMAGES_PATH = Path(TR_FGSM_PATH, 'images')
TR_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/train/')
TR_FGSM_POSTHOC_LABEL_PATH = Path(TR_FGSM_POSTHOC_PATH, 'labels.csv')

VAL_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/validation')
VAL_FGSM_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_FGSM_IMAGES_PATH = Path(VAL_FGSM_PATH, 'images')
VAL_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/validation/')
VAL_FGSM_POSTHOC_LABEL_PATH = Path(VAL_FGSM_POSTHOC_PATH, 'labels.csv')

In [2]:
# If CLASS_SUBSET is specified, INDEX_SUBSET will be ignored. Set CLASS_SUBSET=None if you want to use indexes.
INDEX_SUBSET = get_random_indexes()
CLASS_SUBSET = get_random_classes()
INDEX_SUBSET = None

CLASS_SUBSET = np.load(CLASS_SUBSET_PATH) # for train
#CLASS_SUBSET = None # for validation

BATCH_SIZE = 1

DEVICE = 'cuda'

In [3]:
len(CLASS_SUBSET)

25

In [4]:
#!python $HOME/deeplearning/setup/collect_env.py

## Import DINO
Official repo: https://github.com/facebookresearch/dino

In [5]:
model, linear_classifier = get_dino()

Please use the `--pretrained_weights` argument to indicate the path of the checkpoint to evaluate.
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.
Model vit_small built.
Embed dim 1536
We load the reference pretrained linear weights.


## Dataloaders

In [45]:
tr_ori_loader = create_loader(TR_ORI_IMAGES_PATH, TR_ORI_LABEL_PATH, INDEX_SUBSET, CLASS_SUBSET, BATCH_SIZE, transform=NO_NORM_TRANSFORM)


val_ori_loader = create_loader(VAL_ORI_IMAGES_PATH, VAL_ORI_LABEL_PATH, INDEX_SUBSET, None, BATCH_SIZE, transform=NO_NORM_TRANSFORM)

In [46]:
#dn_loader = create_loader(DN_IMAGES_PATH, DN_LABEL_PATH, INDEX_SUBSET, CLASS_SUBSET, BATCH_SIZE, transform=NO_NORM_TRANSFORM)
val_pgd_loader = create_loader(VAL_PGD_IMAGES_PATH, VAL_PGD_LABEL_PATH, INDEX_SUBSET, None, BATCH_SIZE, transform=TO_TENSOR_TRANSFORM)
val_cw_loader = create_loader(VAL_CW_IMAGES_PATH, VAL_CW_LABEL_PATH, INDEX_SUBSET, None, BATCH_SIZE, transform=TO_TENSOR_TRANSFORM)
tr_fgsm_loader = create_loader(TR_FGSM_IMAGES_PATH, TR_FGSM_LABEL_PATH, INDEX_SUBSET, CLASS_SUBSET, BATCH_SIZE, transform=TO_TENSOR_TRANSFORM)
val_fgsm_loader = create_loader(VAL_FGSM_IMAGES_PATH, VAL_FGSM_LABEL_PATH, INDEX_SUBSET, None, BATCH_SIZE, transform=TO_TENSOR_TRANSFORM)

## Adversarial Sample Tuples

### Iterator

In [47]:
# Performs a forward pass given a sample `inp` and a classifier.
def generate_model_output(inp, n=4):
    inp = inp.to("cuda")
    # add one dimension to input image (get_intermediate_layers expects it)
    inp = inp.unsqueeze(dim=0)
    intermediate_output = model.get_intermediate_layers(inp, n)
    return torch.cat([x[:, 0] for x in intermediate_output], dim=-1)

In [48]:
class AdvTupleIterator:
    def __init__(self, ori_loader, adv_loader, model, linear_classifier, max=0, visualize=False):
        self.samples = adv_dataset(ori_loader, adv_loader, model, linear_classifier)
        self.max = max
        self.num = 0
        self.visualize = visualize
    
    def __iter__(self):
        self.num = 0
        return self
        
    def __next__(self):
        if(self.max > 0 and self.num >= self.max):
            raise StopIteration
        self.num += 1
        # payload can be original or adversarial.
        sample, payload, label = next(self.samples)
        if self.visualize:
            img = payload.permute(1, 2, 0)
            plt.imshow(img.cpu(), interpolation='nearest')
            plt.show()
        payload_out = generate_model_output(payload)
        if self.visualize:
            print(f"shape of image: {img.shape}")
            print(f"shape of model output: {payload_out.shape}")
        return sample, payload_out, label

In [ ]:
# testing iterator
tmp_ori_loader = tr_ori_loader
#tmp_ori_loader = val_ori_loader
#tmp_adv_loader = val_pgd_loader
#tmp_adv_loader = val_cw_loader
#tmp_adv_loader = tr_pgd_loader
tmp_adv_loader = tr_fgsm_loader

total=2
samples = AdvTupleIterator(tmp_ori_loader, tmp_adv_loader, model, linear_classifier, visualize=True)

for i in range(total):
  num, payload, label = next(samples)
  sys.stdout.write(f"\rpayload {i+1}/{total} (label {label})")
  sys.stdout.flush()

### Saving to disk (once per dataset)

In [ ]:
# save to disk!
limit = 10
datasets = [
    #{"posthoc_path": DN_POSTHOC_PATH,  "label_path": DN_POSTHOC_LABEL_PATH},
    {
        "ori_loader": val_ori_loader,
        "adv_loader": val_pgd_loader,
        "posthoc_path": VAL_PGD_POSTHOC_PATH,
        "label_path": VAL_PGD_POSTHOC_LABEL_PATH
    },
    {
        "ori_loader": val_ori_loader,
        "adv_loader": val_cw_loader,
        "posthoc_path": VAL_CW_POSTHOC_PATH,
        "label_path": VAL_CW_POSTHOC_LABEL_PATH
    },
    {
        "ori_loader": val_ori_loader,
        "adv_loader": val_fgsm_loader,
        "posthoc_path": VAL_FGSM_POSTHOC_PATH,
        "label_path": VAL_FGSM_POSTHOC_LABEL_PATH
    },
]

for dataset in datasets:
    ori_loader = dataset["ori_loader"]
    adv_loader = dataset["adv_loader"]
    posthoc_path = dataset["posthoc_path"]
    label_path = dataset["label_path"]

    print(f"posthoc path: {posthoc_path}")
    print(f"label path: {label_path}")

    iterator = AdvTupleIterator(ori_loader, adv_loader, model, linear_classifier, max=0)
    names = []
    paths = []
    labels = []

    for i, (name, payload, label) in enumerate(iterator):
        if limit > 0 and i >= limit: break
        sys.stdout.write(f"\r {i+1}/{limit} {name} (label {label})")
        sys.stdout.flush()
        # original: 0, adversarial: 1
        if label == 0:
            path = f"org/{name}"
        else:
            path = f"adv/{name}"
        paths.append(path)
        names.append(name)
        labels.append(label)
        path = Path(posthoc_path, path)
        torch.save(payload, path)

    df = pd.DataFrame(data={"path": paths, "name": names, "label": labels})
    df.to_csv(label_path, index=False)
    print("")

### Custom dataset for reading from disk

In [50]:
class AdvDataset(torch.utils.data.Dataset):
    def __init__(self, img_folder, index_df):
        super().__init__()
        self.img_folder = img_folder
        self.index_df = index_df
    
    def __len__(self):
        return len(self.index_df)
    
    def __getitem__(self, index):
        filename = self.index_df['path'].iloc[index]
        label = self.index_df['label'].iloc[index]
        payload = torch.load(Path(self.img_folder, filename))
        return filename, payload, label

## Posthoc Classifier

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [52]:
# Simple Binary Classifier Network
class SimpleBC(nn.Module):
    def __init__(self, input_shape):
        super(SimpleBC,self).__init__()
        self.fc1 = nn.Linear(input_shape,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [53]:
# Linear Binary Classifier
class LinearBC(nn.Module):
    def __init__(self, input_shape):
        super(LinearBC,self).__init__()
        self.fc1 = nn.Linear(input_shape,1)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x

In [54]:
from tqdm import tqdm

# Hyperparameters
EPOCHS = 10

# Initialise network
net = LinearBC(1536)

# Select device
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
net.to(device)

# Set model to train
net.train()

# define loss, optimizer, and scheduler
criterion = nn.BCELoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer = optim.Adagrad(net.parameters(), lr=0.01, lr_decay=1e-08, weight_decay=0)
# scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

losses = []
accur = []

train_files = []
test_files = []

def train(train_posthoc_path, val_posthoc_path):
    idx_train_df = pd.read_csv(Path(train_posthoc_path, 'labels.csv'))
    idx_val_df = pd.read_csv(Path(val_posthoc_path, 'labels.csv'))

    #train_dataset, test_dataset = random_split(AdvDataset(posthoc_path, index_df), [train_len, test_len], generator=torch.Generator().manual_seed(42))
    train_dataset = AdvDataset(train_posthoc_path, idx_train_df)
    val_dataset = AdvDataset(val_posthoc_path, idx_val_df)
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=False)
    val_loader = DataLoader(dataset=val_dataset, batch_size=8, shuffle=False)
    
    # Train network
    pbar = tqdm(range(EPOCHS))
    for epoch in pbar:  # loop over the dataset multiple times

        # Metrics
        train_running_loss = 0.0
        train_running_loss_mean = 0.0
        train_acc = 0.0
        train_acc_mean = 0.0
        test_running_loss = 0.0
        test_acc = 0.0

        for i, (filename, inputs, labels) in enumerate(train_loader, start=0):        
            train_files.append(filename)
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            # Forward Pass
            outputs = net(inputs).float()
            outputs = outputs.reshape(-1)

            # Backpropagation
            optimizer.zero_grad() # Reset the gradient
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # loss train
            train_running_loss += loss.item()
            train_running_loss_mean = train_running_loss / (i+1)

            # accuracy train
            predicted = net(inputs).reshape(-1).detach().cpu().numpy().round()
            acc_labels = labels
            acc_labels = acc_labels.detach().cpu().numpy()
            inter = np.equal(predicted, acc_labels)
            train_acc += inter.sum()
            #train_acc_mean = train_acc / (i+1)

        with torch.no_grad():
            for filename, inputs, labels in val_loader:
                test_files.append(filename)
                try:
                    # get the inputs; data is a list of [inputs, labels] and write to device
                    inputs = inputs.to(device)
                    labels = labels.to(device).float()

                    # Forward Pass
                    outputs = net(inputs).float()
                    outputs = outputs.reshape(-1)

                    # loss test
                    loss = criterion(outputs, labels)
                    test_running_loss += loss.item()

                    # accuracy test
                    outputs = outputs.detach().cpu().numpy().round()
                    comparison = np.equal(labels.detach().cpu().numpy(), outputs)
                    test_acc += comparison.sum()

                except Exception as e:
                    print("Error: {}".format(e))
                    pass

        losses.append(train_running_loss_mean)
        accur.append(train_acc_mean)
        pbar.set_description("Ep: {}\t Tr. Loss: {:.4f}\t Tr. Acc: {:.4f}\t T. Loss: {:.4f}\t T. Acc: {:.4f}".format(epoch, 
                                                                                train_running_loss_mean, 
                                                                                train_acc / len(train_loader.dataset), 
                                                                                test_running_loss, 
                                                                                test_acc / len(val_loader.dataset)))

    print('Finished Training')

#print("DAmageNet:")
#train(DN_POSTHOC_PATH)
print("PGD:")
train(TR_PGD_POSTHOC_PATH, VAL_PGD_POSTHOC_PATH)
print("CW:")
train(VAL_CW_POSTHOC_PATH, VAL_CW_POSTHOC_PATH)
print("FGSM:")
train(VAL_FGSM_POSTHOC_PATH, VAL_FGSM_POSTHOC_PATH)

PGD:


NameError: name 'TR_PGD_POSTHOC_PATH' is not defined